## Importation des bliblothèques

In [2]:
import sqlite3
import pandas as pd

## Connection à la base de données

In [3]:
connect = sqlite3.connect("../../Databases/raw-database.db")
cursor = connect.cursor()

## Importation des données freeze frames

In [4]:
req = cursor.execute(f"SELECT * FROM freeze_frames")
res = req.fetchall()
desc = req.description
freeze_frames = pd.DataFrame(res)
freeze_frames.columns = [i[0] for i in desc]

In [5]:
# Aperçu des données
freeze_frames.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
0,4820,2025-01-10 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN
1,4834,2025-01-10 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0
2,4854,2025-01-10 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN


## Importation des données events

In [6]:
# On récupère les event_id SB pour pouvoir importer uniquement ceux qui nous intéressent depuis la table events
event_id_freeze_frames = freeze_frames.event_id.unique().tolist()

In [7]:
req = cursor.execute(f"SELECT event_id, pass_cross, pass_type FROM events WHERE event_id IN ({', '.join('?' * len(event_id_freeze_frames))})",
                     event_id_freeze_frames)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [8]:
# Aperçu des données
events.head(3)

,event_id,pass_cross,pass_type
0,2e85ad3e-59c4-479a-a478-4c98174f686b,NaN,None
1,872df8f3-a750-4b61-8890-61a35e96b4a4,1.0,None
2,0f842e18-1363-4af3-8faa-6c9cce55b118,NaN,None


## Jointure des events avec les freeze frames

In [9]:
freeze_frames_events = pd.merge(freeze_frames, events, how = "left", on = "event_id")
freeze_frames_events.sort_values(by = ["match_id_SKC", "timestamp"], inplace = True)

In [10]:
# Aperçu des données
freeze_frames_events.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type
0,4820,2025-01-10 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,NaN,None
1,4834,2025-01-10 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0,1.0,None
2,4854,2025-01-10 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN,NaN,None


## Analyse des centres

In [11]:
# Création d'un dataframe ne comprenant que les centres du dataframe initial
centres = freeze_frames_events[freeze_frames_events.pass_cross == 1]

In [12]:
# On regarde si il y'a des valeurs centres qui sont présents plusieurs fois
# On utilise l'identifiant ["frame", "match_id_SKC"] qui permet d'identifier, de manière unique, chaque frame des données
centres[["frame", "match_id_SKC"]].duplicated().any()

np.False_

On souhaite maintenant identifier chaque centre et chaque évènement précédant un centre par un centre_id.  
Chaque couple (event précédant le centre, centre) sera alors identifié par un unique centre_id.  

De plus, on part du principe qu'on dispose bien de chaque évènement précédant les centres.  
En effet, étant donné que nous ne nous sommes pas chargé de fusionner les évènements précédant et suivant les centres aux évènements correspondant aux centres, nous ne pouvons pas gérer ces erreurs.

Pour gérer les cas ou l'évènement suivant un centre (centre1) est également un centre (centre2), nous partons du principe que ces centres ne sont pas dupliqués mais plutôt que ces centres se suivent en terme de frame.  
En effet, nous n'aurons pas 2 couples (event avant centre1, centre1, centre2) et (centre1, centre2, event après centre2) mais plutot un couple (event avant centre1, centre1, centre2, event après centre2)

In [13]:
# Nous partons de ce principe car nous n'avons pas de valeur dupliquée pour la colonne event_id :
print(freeze_frames_events.event_id.is_unique)

True


Enfin, il est nécessaire des supprimer les events correspondants à des coups de pied arretés car ce n'est pas cohérent d'étudier les évènements précédant des CPA.

In [14]:
# Liste des CPA définis par Stats Bomb
CPA = ["Corner", "Free Kick", "Goal Kick", "Kick Off"]

# Supression des CPA des centres
centres = centres[~centres.pass_type.isin(CPA)]

In [15]:
# identification des évènements précédant les centres
index_events_avant_centre = centres.index - 1

In [16]:
# Création d'un dataframe ne comprenant que les actions précédents les centres
events_avant_centre = freeze_frames_events.loc[index_events_avant_centre]

In [17]:
# On regarde si il y'a des évènements précédents des centres qui correspondent à des centres
events_avant_centre[events_avant_centre.pass_cross == 1]

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type


In [18]:
# Autre moyen de vérifier
print(len(index_events_avant_centre.intersection(centres.index)))

0


Il n'y en a pas

In [19]:
# Attribution des centre_id à chaque events précédant les centres et à chaque centre
events_avant_centre["centre_id"] = range(len(events_avant_centre))
centres["centre_id"] = range(len(events_avant_centre))

In [38]:
# On attribue aux events précédant les centres l'équipe qui a effectué le centre (suivant ces events)
# En effet, il est possible que l'équipe en possession du ballon lors de l'évènement précédant un centre ne soit pas la même
# que celle qui a effectué le centre en question
# De ce fait, il est nécessaire de remplacer l'équipe actuellement attribuée aux events précédant les centres par les équipes ayant effectué 
# les centres
events_avant_centre.drop("group", axis = 1, inplace = True)
events_avant_centre = pd.merge(events_avant_centre, centres[["group", "centre_id"]], on = "centre_id")

# De plus, pour éviter les confusions, nous allons renomer la colonne "group" par "group_centre"
events_avant_centre.rename({"group" : "group_centre"}, axis = 1, inplace = True)

In [39]:
events_avant_centre.head(2)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,tackable_object,pass_cross,pass_type,centre_id,group_centre
0,4820,2025-01-10 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,NaN,NaN,None,0,away team
1,5752,2025-01-10 00:06:02.200000,1.0,c1a7a598-9690-4a32-a733-b7b3b585df83,-43.4,-27.455,1,1020089,NaN,NaN,None,1,None


Nous avons terminé cette première étape qui nous a permis d'analyser et extraire les centres ainsi que les évènements les précédant.  
Nous obtenons finalement un dataframe composé de l'ensemble des events précédant les centres dont nous disposons.  
Ce dataframe contient des informations cruciales telles que la position de l'event ("event_x" et "event_y"), l'identifiant de la frame (le couple ["frame", "match_id_SKC"]) ou encore l'identifiant du centre que nous avons créé, dans le but de pouvoir retrouver le centre auquel correspond l'event précédant un centre.

Nous allons maintenant passer à l'analyse de la position des joueurs au moment de l'event précédant le centre.  
Pour cela, nous allons utiliser les données freeze frames data que nous avions au préalable importées dans la BDD SQLite grâce à nos programmes d'importation.

# Importation des données freeze frames data

In [40]:
req = cursor.execute(f"SELECT trackable_object, x, y, frame, match_id_SKC FROM freeze_frames_data")
res = req.fetchall()
desc = req.description
freeze_frames_data = pd.DataFrame(res)
freeze_frames_data.columns = [i[0] for i in desc]

In [54]:
# Aperçu des données
freeze_frames_data.head(2)

,trackable_object,x,y,frame,match_id_SKC
0,12235.0,45.43,-0.97,4820,1020089
1,12327.0,32.26,-2.78,4820,1020089


Nous allons maintenant fusionner le dataframe des events précédant les centres avec le dataframe des freeze frames data.  
En effet, on souhaite étudier seulement les positions des joueurs sur les events précédant des centres, or le dataframe freeze_frames_data actuel contient les données de toutes les frames disponibles.  
Pour effectuer ce merge, nous allons joindre les deux dataframes par rapport au couple ["match_id_SKC", "frame"] qui identifie de manière unique chaque frame.

In [43]:
ffd_avant_centre = pd.merge(events_avant_centre[["match_id_SKC", "frame", "centre_id", "group_centre"]],
                              freeze_frames_data, on = ["match_id_SKC", "frame"])

In [44]:
ffd_avant_centre.head(2)

,match_id_SKC,frame,centre_id,group_centre,trackable_object,x,y
0,1020089,4820,0,away team,12235.0,45.43,-0.97
1,1020089,4820,0,away team,12327.0,32.26,-2.78


Nous allons maintenant regarder s'il y'a des frames (des events précédant les centres) pour lesquelles nous n'avons pas de tracking data

In [154]:
print("Nombre de centres pour lesquels nous n'avons pas d'information sur la frame de l'event précédent le centre dans les données freeze frames data :", len(events_avant_centre[~events_avant_centre.centre_id.isin(freeze_frames_data.centre_id)]))
print("Nombre total de frames correspondant à des events précédant un centre :", len(events_avant_centre))

Nombre de centres pour lesquels nous avons pas d'information sur la frame de l'event précédent le centre dans les données freeze frames data : 38
Nombre total de frames correspondant à des events précédant un centre : 6181


On remarque qu'il y'a seulement 38 des 6181 frames pour lesquelles nous n'avons pas de données de tracking, ce qui reste raisonnablement faible.

---

Rappel : nous souhaitons étudier le nombre d'adversaires entre le ballon et le but pendant l'évènement précédant le centre.  
Or, pour l'instant, il nous manque les informations suivantes :
- Les id Skill Corner des équipes (à domicile et extérieur pour chaque match)
- les équipes auxquels appartiennent les joueurs  

De ce fait, nous devons obtenir pour chaque match, les team id Skill Corner des équipes à domicile et à l'extérieure, ainsi que pour chaque joueur, l'équipe à laquelle ils appartiennent.  
C'est ce que nous allons faire dans les deux prochaines sections.  
Il est important de noté que nous ne disposons pas directement de ces informations, nous allons donc devoir manipuler les différentes données de matching (entre les id Stats Bomb et Skill Corner).

# Matching des équipes (domicile et extérieur)

Pour obtenir ces informations, nous allons nous servir de des données dont nous disposons actuellement :
- Nous avons les matching des id Skill Corner et Stats Bomb des équipes
- De même pour les id des matches
- Enfin, grâce aux données matches de Stats Bomb (SB_matches), nous savons pour chaque match quelle était l'équipe à domicile et quelle était l'équipe à l'extérieur

En effectuant diverses jointures entre ces tables, nous allons réussir à retrouver les informations souhaitées pour les tracking data.

In [55]:
# Importation des données de matching entre les id Skill Corner et Stats Bomb des matches
req = cursor.execute(f"SELECT * FROM matching_matches")
res = req.fetchall()
desc = req.description
matching_matches = pd.DataFrame(res)
matching_matches.columns = [i[0] for i in desc]

In [56]:
matching_matches.head(2)

,match_id_SB,match_id_SKC
0,3894366.0,1547880
1,3894367.0,1547881


In [47]:
# Importation des données de matching entre les id Skill Corner et Stats Bomb des équipes
req = cursor.execute(f"SELECT * FROM matching_teams")
res = req.fetchall()
desc = req.description
matching_teams = pd.DataFrame(res)
matching_teams.columns = [i[0] for i in desc]

In [48]:
matching_teams.head(2)

,team_id_SB,team_id_SKC
0,168,85
1,144,66


In [49]:
# Importation des informations Stats Bomb sur l'ensemble des matches dont nous disposons.
# Pour éviter les confusions
req = cursor.execute(f"SELECT match_id_SB, home_team_id_SB, away_team_id_SB FROM SB_matches")
res = req.fetchall()
desc = req.description
SB_matches = pd.DataFrame(res)
SB_matches.columns = [i[0] for i in desc]

In [50]:
SB_matches.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB
0,3894290,136,139
1,3894367,165,152


In [51]:
merge1 = pd.merge(SB_matches, matching_matches, on = "match_id_SB")

In [52]:
merge2 = pd.merge(merge1, matching_teams, left_on = "home_team_id_SB", right_on = "team_id_SB")
merge2 = merge2.drop("team_id_SB", axis = 1).rename({"team_id_SKC" : "home_team_id_SKC"}, axis = 1)
merge2.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,home_team_id_SKC
0,3894290,136,139,1434956,70
1,3894367,165,152,1547881,72


In [53]:
merge3 = pd.merge(merge2, matching_teams, left_on = "away_team_id_SB", right_on = "team_id_SB")
merge3 = merge3.drop("team_id_SB", axis = 1).rename({"team_id_SKC" : "away_team_id_SKC"}, axis = 1)
merge3.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,home_team_id_SKC,away_team_id_SKC
0,3894290,136,139,1434956,70,86
1,3894367,165,152,1547881,72,328
